In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score
import re

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train['text'][40:]

,text
40,Check these out: http://t.co/rOI2NSmEJJ http:/...
41,on the outside you're ablaze and alive\nbut yo...
42,Had an awesome time visiting the CFC head offi...
43,SOOOO PUMPED FOR ABLAZE ???? @southridgelife
44,I wanted to set Chicago ablaze with my preachi...
...,...
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,Police investigating after an e-bike collided ...


In [6]:
def preprocess_data(text):
  text = text.lower()
  text = re.sub(r'http\S+','',text)
  text = re.sub(r'[^a-zA-Z0-9\s]','',text)
  return text

In [7]:
train['text'] = train['text'].apply(preprocess_data)
test['text'] = test['text'].apply(preprocess_data)

In [8]:
train_text, val_texts, train_labels, val_labels = train_test_split(train['text'].tolist(),train['target'].tolist(),test_size = 0.2,random_state = 42)

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels = 2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train_encodings = tokenizer(train_text, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)
test_encodings = tokenizer(test['text'].tolist(), truncation = True, padding = True)

In [11]:
train_data = TensorDataset(torch.tensor(train_encodings['input_ids']),
                           torch.tensor(train_encodings['attention_mask']),
                           torch.tensor(train_labels))
val_data = TensorDataset(torch.tensor(val_encodings['input_ids']),
                           torch.tensor(val_encodings['attention_mask']),
                           torch.tensor(val_labels))
test_data = TensorDataset(torch.tensor(test_encodings['input_ids']),
                           torch.tensor(test_encodings['attention_mask']))

In [12]:
train_dataloader = DataLoader(train_data,sampler = RandomSampler(train_data), batch_size = 32)
val_dataloader = DataLoader(val_data,sampler = RandomSampler(val_data), batch_size = 32)
test_dataloader = DataLoader(test_data,sampler = SequentialSampler(test_data), batch_size = 32)

In [13]:
train_encodings['input_ids'][0]

[101,
 26103,
 1998,
 7481,
 4106,
 1997,
 2342,
 2000,
 2224,
 9593,
 5968,
 1999,
 3386,
 20168,
 19841,
 2887,
 2510,
 4188,
 7806,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [14]:
optimizer = AdamW(model.parameters(),lr = 2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
device = torch.device("cuda")

In [16]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
for epoch in range(3):
  model.train()
  for batch in train_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_mask, labels = batch
    optimizer.zero_grad()
    outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    scheduler.step()

  model.eval()
  val_predictions = []
  with torch.no_grad():
    for batch in val_dataloader:
      batch = tuple(t.to(device) for t in batch)
      input_ids, attention_mask, labels = batch
      outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=1).tolist()
      val_predictions.extend(predictions)
  val_f1 = f1_score(val_labels, val_predictions)
  print(f'Epoch {epoch+1} - Validation F1: {val_f1}')

Epoch 1 - Validation F1: 0.3672413793103448
Epoch 2 - Validation F1: 0.370668815471394
Epoch 3 - Validation F1: 0.39259855189058723


In [21]:
model.eval()
test_predcitions = []
with torch.no_grad():
  for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_mask = batch
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    test_predcitions.extend(predictions)

#Save the predictions
submissions_df = pd.DataFrame({'id': test['id'],'target': test_predcitions})
submissions_df.to_csv("submissions.csv",index = False)